In [98]:
import csv 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime


In [99]:
data = csv.reader(open('.\\clean\\fire_incidents-csv-small.csv'))



In [100]:
data = np.array(list(data))
data = data[1:]


#print(list(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') for x in data[:,6]))
for i in [6,7]:
    data[:,i] = list(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S') for x in data[:,i])





In [116]:
print(list((np.datetime64(x[7]) - np.datetime64(x[6])).astype('timedelta64[s]').astype(np.int32) for x in data))


[522, 338, 197, 287, 456, 401, 154, 127, 295, 384, 317, 640, 271, 262, 379, 680, 582, 443, 465, 301, 311, 234, 279, 249, 362, 241, 330, 170, 316, 361, 711, 196, 412, 516, 643, 511, 962, 376, 436, 469, 179, 438, 630, 232, 241, 165, 275, 154, 530, 315, 371, 284, 234, 252, 224, 123, 0, 295, 391, 541, 305, 261, 241, 151, 349, 194, 397, 189, 256, 348, 316, 204, 136, 196, 160, 169, 170, 161, 129, 235, 198, 119, 212, 147, 232, 162, 189, 146, 418, 256, 177, 198, 180, 266, 236, 133, 295, 102, 120, 1853]


In [55]:
print(BBox)

(-122.509368, -122.3748374, 37.616901, 37.83019284)
